In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras import optimizers
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
img_rows, img_cols = 28, 28

In [3]:
num_classes = 10

In [4]:
d_train = pd.read_csv('data/train.csv')

In [5]:
pixels = d_train.columns.drop('label')
y_column = 'label'

In [6]:
d_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Нормализация

In [7]:
d_train[pixels] = d_train[pixels].applymap(lambda x: x / 255)

In [8]:
train, validation = train_test_split(d_train, test_size=0.1)

In [9]:
mat_train = train[pixels].as_matrix()
x_train = mat_train.reshape(mat_train.shape[0], img_rows, img_cols, 1)
y_train = train[y_column].as_matrix()

mat_val = validation[pixels].as_matrix()
x_val = mat_val.reshape(mat_val.shape[0], img_rows, img_cols, 1)
y_val = validation[y_column].as_matrix()

In [10]:
y_train = y_train.reshape((y_train.shape[0], 1))
y_val = y_val.reshape((y_val.shape[0], 1))

In [11]:
y_train = np_utils.to_categorical(y_train, 10)
y_val = np_utils.to_categorical(y_val, 10)

In [12]:
print(x_train.shape, y_train.shape)

(37800, 28, 28, 1) (37800, 10)


In [13]:
# Original NN
# model = Sequential()
# model.add(Dense(units=600, activation='relu', input_dim=len(pixels)))
# model.add(Dense(units=80, activation='relu'))
# model.add(Dense(units=10, activation='softmax'))
# sgd = optimizers.SGD(lr=0.08,nesterov=True)
# model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [14]:
input_shape = (x_train.shape[1], x_train.shape[2], 1)
input_shape

(28, 28, 1)

In [15]:
epochs_count = 14

In [16]:
# Classic CNN
model = Sequential()
model.add(Conv2D(75, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(100, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs_count, batch_size=16)

Train on 37800 samples, validate on 4200 samples
Epoch 1/14
37800/37800 [==============================] - 195s 5ms/step - loss: 0.1598 - acc: 0.9496 - val_loss: 0.0692 - val_acc: 0.9776
Epoch 2/14
37800/37800 [==============================] - 192s 5ms/step - loss: 0.0623 - acc: 0.9805 - val_loss: 0.0390 - val_acc: 0.9888
Epoch 3/14
37800/37800 [==============================] - 194s 5ms/step - loss: 0.0515 - acc: 0.9840 - val_loss: 0.0471 - val_acc: 0.9862
Epoch 4/14
37800/37800 [==============================] - 159s 4ms/step - loss: 0.0411 - acc: 0.9879 - val_loss: 0.0512 - val_acc: 0.9860
Epoch 5/14
37800/37800 [==============================] - 159s 4ms/step - loss: 0.0411 - acc: 0.9881 - val_loss: 0.0365 - val_acc: 0.9902
Epoch 6/14
37800/37800 [==============================] - 159s 4ms/step - loss: 0.0340 - acc: 0.9898 - val_loss: 0.0461 - val_acc: 0.9893
Epoch 7/14
37800/37800 [==============================] - 157s 4ms/step - loss: 0.0354 - acc: 0.9900 - val_loss: 0.0440 - v

In [19]:
model.save('model_cnn_'+str(epochs_count)+'.h5')

In [20]:
# На критический случай
#del model
#model = load_model('model_10.h5')

In [21]:
score = model.evaluate(x_val, y_val,verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.08690047648
Test accuracy: 0.986904761905


In [22]:
dtest = pd.read_csv('data/test.csv', sep=',')
mat_test = dtest.as_matrix()
mat_test = mat_test.reshape(dtest.shape[0], img_rows, img_cols, 1)
y_test_predict = model.predict(mat_test)
y_test_predict = np.argmax(y_test_predict,axis=1)

In [23]:
result = pd.DataFrame({'Label': []})

In [24]:
result['Label'] = y_test_predict
result.index += 1 
result.index.name = 'ImageId'
result.to_csv('data/Result_cnn_'+str(epochs_count)+'.csv')
result.head()

,Label
ImageId,
1,2
2,0
3,9
4,9
5,3
